The company of this exercise is a social network. They decided to add a feature called: Recommended Friends, i.e. they suggest people you may know.

A data scientist has built a model to suggest 5 people to each user. These potential friends will be shown on the user newsfeed. At ﬁrst, the model is tested just on a random subset of users to see how it performs compared to the newsfeed without the new feature.

The test has been running for some time and your boss asks you to check the results. You are asked to check, for each user, the number of pages visited during their ﬁrst session since the test started. If this number increased, the test is a success.

# Index
* [Answer question 1](#Answer-question-1)
* [Answer question 2 and 3](#Answer-question-2-and-3)
    * [Browsers' impact](#Browsers'-impact)
    * [First time impact](#First-time-impact)
    * [Browser and 'First time' combined impact](#Browser-and-'First-time'-combined-impact)

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss

# Answer question 1
<span style='color:blue'>Is the test winning? That is, should 100% of the users see the Recommended Friends feature?</span>

In [2]:
tests = pd.read_csv("test_table.csv",index_col='user_id')
users = pd.read_csv("user_table.csv",index_col='user_id')

tests = tests.join(users)

tests['date'] = pd.to_datetime(tests.date)
tests['signup_date'] = pd.to_datetime(tests.signup_date)

In [3]:
tests.head()#glance

,date,browser,test,pages_visited,signup_date
user_id,,,,,
600597,2015-08-13,IE,0,2,2015-01-19
4410028,2015-08-26,Chrome,1,5,2015-05-11
6004777,2015-08-17,Chrome,0,8,2015-06-26
5990330,2015-08-27,Safari,0,8,2015-06-25
3622310,2015-08-07,Firefox,0,1,2015-04-17


In [6]:
def run_ttest(df):
    vp_in_test = df.loc[tests.test == 1, 'pages_visited']
    test_mean = vp_in_test.mean()
    
    vp_in_ctrl = df.loc[tests.test == 0, 'pages_visited']
    ctrl_mean = vp_in_ctrl.mean()
    
    result = ss.ttest_ind(vp_in_ctrl, vp_in_test, equal_var=False)
    conclusion = 'Significant' if result.pvalue < 0.05 else 'Not Significant'
    
    return pd.Series({'n_test':vp_in_test.shape[0],
                      'n_ctrl': vp_in_ctrl.shape[0],
                      'mean_test': test_mean,
                      'mean_ctrl': ctrl_mean,
                      'test-ctrl': test_mean - ctrl_mean,
                      'pvalue':result.pvalue,
                      'conclusion':conclusion})

In [7]:
run_ttest(tests)

conclusion    Not Significant
mean_ctrl             4.60839
mean_test             4.59969
n_ctrl                  49846
n_test                  50154
pvalue               0.577452
test-ctrl         -0.00870091
dtype: object

according to above Hypothesis Test result, there is ** no significant improvement in test group **. 

# Answer question 2 and 3
* <span style='color:blue'>Is the test performing similarly for all user segments or are there diﬀerences among diﬀerent segments?</span>
* <span style='color:blue'>If you identiﬁed segments that responded diﬀerently to the test, can you guess the reason? Would this change your point 1 conclusions?</span>

In [8]:
tests['n_days_after_sign'] = (tests.date - tests.signup_date).dt.days
tests['first_time'] = (tests.n_days_after_sign == 0).astype(int)

## Browsers' impact

In [10]:
tests.groupby('browser').apply(run_ttest)

,conclusion,mean_ctrl,mean_test,n_ctrl,n_test,pvalue,test-ctrl
browser,,,,,,,
Chrome,Significant,4.613341,4.690680,21453,21974,9.434084e-04,0.077339
Firefox,Significant,4.600164,4.714259,10972,10786,5.817199e-04,0.114095
IE,Significant,4.598478,4.685985,10906,10974,7.829509e-03,0.087507
Opera,Significant,4.546438,0.000000,1109,1018,2.252939e-321,-4.546438
Safari,Not Significant,4.638180,4.692336,5406,5402,2.411738e-01,0.054156


from above result, we can see that, by applying this "Recommend Friend" feature
* #page_visited in Chrome, Firefox, IE are significantly increased.
* #page_visited in Opera has reduced to zero, <span style='color:orange;font-size:1.2em'>maybe there is some bug in implementation on Opera, which stops user visiting further pages.</span>
* #page_visited in Safari has no significant improvement, <span style='color:orange;font-size:1.2em'>maybe because the recommended friends aren't shown in a noticeable position.</span>

## First time impact

In [11]:
tests.groupby('first_time').apply(run_ttest)

,conclusion,mean_ctrl,mean_test,n_ctrl,n_test,pvalue,test-ctrl
first_time,,,,,,,
0,Not Significant,4.603284,4.622379,39890,40109,0.261837,0.019095
1,Significant,4.628867,4.509109,9956,10045,0.001742,-0.119758


above result shows:
* for old users, the new feature improve #page_visted, but the change isn't significant
* for first-time new users, ** this new feature significantly decrease #page_visited **.

this is a strange observation. Since I already suspect there is <span style='color:red;font-size:1.2em'>some bug in Opera's implementation (which reduce the visited number to 0 after using this new feature)</span>, so I need to split the dataset further by browser.

## Browser and 'First time' combined impact

In [13]:
ttest_result = tests.groupby(['browser','first_time']).apply(run_ttest)

In [14]:
ttest_result

conclusion  mean_ctrl  mean_test  n_ctrl  n_test  \
browser first_time                                                          
Chrome  0               Significant   4.607945   4.701512   17092   17525   
        1           Not Significant   4.634488   4.648011    4361    4449   
Firefox 0               Significant   4.590590   4.757306    8842    8657   
        1           Not Significant   4.639906   4.539220    2130    2129   
IE      0               Significant   4.590576   4.721494    8744    8779   
        1           Not Significant   4.630435   4.543964    2162    2195   
Opera   0               Significant   4.594564   0.000000     883     833   
        1               Significant   4.358407   0.000000     226     185   
Safari  0           Not Significant   4.638254   4.720973    4329    4315   
        1           Not Significant   4.637883   4.578657    1077    1087   

                           pvalue  test-ctrl  
browser first_time                            
Chrome  0            2.290889e-04   0.093567  
        1            8.149175e-01   0.013523  
Firefox 0            3.692901e-06   0.166716  
        1            2.210706e-01  -0.100686  
IE      0            2.669847e-04   0.130918  
        1            2.808421e-01  -0.086471  
Opera   0           7.204927e-255  -4.594564  
        1            1.222949e-68  -4.358407  
Safari  0            1.000829e-01   0.082720  
        1            6.015241e-01  -0.059226

In [16]:
# old users on each browser
ttest_result.xs(0,level=1)

,conclusion,mean_ctrl,mean_test,n_ctrl,n_test,pvalue,test-ctrl
browser,,,,,,,
Chrome,Significant,4.607945,4.701512,17092,17525,2.290889e-04,0.093567
Firefox,Significant,4.590590,4.757306,8842,8657,3.692901e-06,0.166716
IE,Significant,4.590576,4.721494,8744,8779,2.669847e-04,0.130918
Opera,Significant,4.594564,0.000000,883,833,7.204927e-255,-4.594564
Safari,Not Significant,4.638254,4.720973,4329,4315,1.000829e-01,0.082720


for old users, the conclusion is the same as general ['browser impact'](#Browsers'-impact), which is:
* #page_visited in Chrome, Firefox, IE are significantly increased.
* #page_visited in Opera has reduced to zero, <span style='color:orange;font-size:1.2em'>maybe there is some bug in implementation on Opera, which stops user visiting further pages.</span>
* #page_visited in Safari has no significant improvement, <span style='color:orange;font-size:1.2em'>maybe because the recommended friends aren't shown in a noticeable position.</span>

In [17]:
# first-time new users on each browser
ttest_result.xs(1,level=1)

,conclusion,mean_ctrl,mean_test,n_ctrl,n_test,pvalue,test-ctrl
browser,,,,,,,
Chrome,Not Significant,4.634488,4.648011,4361,4449,8.149175e-01,0.013523
Firefox,Not Significant,4.639906,4.539220,2130,2129,2.210706e-01,-0.100686
IE,Not Significant,4.630435,4.543964,2162,2195,2.808421e-01,-0.086471
Opera,Significant,4.358407,0.000000,226,185,1.222949e-68,-4.358407
Safari,Not Significant,4.637883,4.578657,1077,1087,6.015241e-01,-0.059226


besides Opera which may have a bug, all changes for new users are not significant, and there is even some drop after applying this new feature.

this may because: <span style='color:orange'> the friend recommendation engine may be based on a user's previous social activity on the site. then for new users, since they don't have any previous history for recommendation engine to use, the recommendation result is like random guess, which cannot draw new user's interest. But the recommended friends may occupy some space on the page, so it may even decrease the #page_visited for new users a little bit.</span>

from this observation, I suspect the Recommedation Engine suffers <span style='color:red;font-size:1.2em;font-weight:bold'>'the cold start'</span> problem.